# 3.Feature Engineering

Import data

In [2]:
import pandas as pd
import numpy as np
import sklearn 
from sklearn.decomposition import PCA
import scipy.stats as ss
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import cross_val_score,cross_val_predict
from sklearn import svm
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, mean_absolute_error,r2_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor 
import datetime as dt
import copy

Create preprocessing functions

In [3]:
class preProcessing:
    def isNull(frame):
        null_count_ser = pd.isnull(frame).sum()
        is_null_ser = null_count_ser > 0
        return null_count_ser[is_null_ser]
    def uniqueValues(frame):
        for col in frame.columns:
            print(col, frame[col].dtype, "has", len(frame[col].unique()),"unique values")
    def dummiesColumns(frame,int):
        dummiesList=[]
        for col in frame.columns:
            if len(frame[col].unique())<= int:
                dummiesList.append(col)
        return dummiesList
    def getDummies(frame1,frame2,int):
        dummiesList=[]
        for col in frame2.columns:
            if len(frame2[col].unique())<= int:
                dummiesList.append(col)
        frame3=pd.get_dummies(frame1[dummiesList])
        return frame3
    def corrColumns(frame):
        df_corr=frame.corr()
        return df_corr
    def cramers_v(x, y):
        confusion_matrix = pd.crosstab(x,y)
        chi2 = ss.chi2_contingency(confusion_matrix)[0]
        n = confusion_matrix.sum().sum()
        phi2 = chi2/n
        r,k = confusion_matrix.shape
        phi2corr = max(0, phi2-((k-1)*(r-1))/(n-1))
        rcorr = r-((r-1)**2)/(n-1)
        kcorr = k-((k-1)**2)/(n-1)
        return np.sqrt(phi2corr/min((kcorr-1),(rcorr-1)))

# 3.1 Customers.csv

## 3.1.1 Basic info

In [36]:
df_customers=pd.read_csv('customers.csv',header=0)
df_customers_original=pd.read_csv('customers.csv',header=0)
print(df_customers.shape)
print(preProcessing.uniqueValues(df_customers))
df_customers.head(10)

(1371980, 7)
customer_id object has 1371980 unique values
FN float64 has 2 unique values
Active float64 has 2 unique values
club_member_status object has 4 unique values
fashion_news_frequency object has 5 unique values
age float64 has 85 unique values
postal_code object has 352899 unique values
None


,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...
5,000064249685c11552da43ef22a5030f35a147f723d5b0...,NaN,NaN,NaN,NaN,NaN,2c29ae653a9282cce4151bd87643c907644e09541abc28...
6,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,NaN,NaN,ACTIVE,NONE,20.0,fe7b8e2b3fafb89ca90db17ffeeae0fd29b795d803f749...
7,00007d2de826758b65a93dd24ce629ed66842531df6699...,1.0,1.0,ACTIVE,Regularly,32.0,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...
8,00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77...,NaN,NaN,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...
9,00008469a21b50b3d147c97135e25b4201a8c58997f787...,NaN,NaN,ACTIVE,NONE,20.0,2c29ae653a9282cce4151bd87643c907644e09541abc28...


* drop column 'postal_code', since its unique values is 1/3 of those of primary key. 
* and it won't offer useful features.

In [37]:
df_customers.drop('postal_code', axis=1, inplace=True)

In [38]:
print(df_customers['FN'].unique())
print(df_customers['Active'].unique())
print(df_customers['club_member_status'].unique())
print(df_customers['fashion_news_frequency'].unique())

[nan  1.]
[nan  1.]
['ACTIVE' nan 'PRE-CREATE' 'LEFT CLUB']
['NONE' 'Regularly' nan 'Monthly' 'None']


### 3.1.2 Null values

In [39]:
preProcessing.isNull(df_customers)

FN                        895050
Active                    907576
club_member_status          6062
fashion_news_frequency     16009
age                        15861
dtype: int64

* too many null values for categorical columns 'FN', 'Active','club_member_status','fashion_news_frequency'.
* fill them with 'NONE'.

In [40]:
#fill 'NONE' for all the null cells in categorical columns
df_customers['FN'].replace('','NONE',inplace=True)
df_customers['Active'].replace('','NONE',inplace=True)
df_customers['club_member_status'].replace('','NONE',inplace=True)
df_customers['club_member_status'].replace('nan','NONE',inplace=True)
df_customers['club_member_status'].replace('NaN','NONE',inplace=True)
df_customers['fashion_news_frequency'].replace('','NONE',inplace=True)
df_customers['fashion_news_frequency'].replace('NaN','NONE',inplace=True)
df_customers['fashion_news_frequency'].replace('nan','NONE',inplace=True)
df_customers['fashion_news_frequency'].replace('None','NONE',inplace=True)

df_customers.head(5)

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0


### 3.1.3 Delete highly correlated columns

* it seems that 3 columns 'FN', 'Active','club_member_status','fashion_news_frequency' are highly correlated.
* compute their pearson correlation rate

In [41]:
df_customers_corr=pd.DataFrame()
df_customers_corr['FN_Category']=df_customers['FN'].astype('category').cat.codes
df_customers_corr['Active_Category']=df_customers['Active'].astype('category').cat.codes
df_customers_corr['club_member_status_Category']=df_customers['club_member_status'].astype('category').cat.codes
df_customers_corr['fashion_news_frequency_Category']=df_customers['fashion_news_frequency'].astype('category').cat.codes
df_customers_corr['age_numerical']=df_customers['age']
df_customers_corr.corr()

,FN_Category,Active_Category,club_member_status_Category,fashion_news_frequency_Category,age_numerical
FN_Category,1.000000,0.979947,-0.154874,0.908909,0.033571
Active_Category,0.979947,1.000000,-0.151753,0.891508,0.036474
club_member_status_Category,-0.154874,-0.151753,1.000000,-0.136776,0.080077
fashion_news_frequency_Category,0.908909,0.891508,-0.136776,1.000000,0.025478
age_numerical,0.033571,0.036474,0.080077,0.025478,1.000000


* delete column 'FN', 'Active', since the corr rate is more than 95% among them.

In [42]:
df_customers.drop('FN', axis=1, inplace=True)
df_customers.drop('Active', axis=1, inplace=True)
df_customers_forFinalMerge=df_customers

### 3.1.4 Fill nulls for column 'age'

one-hot encoding for categorical columns
* we later will utilize the tool Recbole to do prediction, and it will implement the one-hot internally for us.
* here we do the one-hot just for filling nulls for column 'age', since it's numerical, we will implement Linear Regression and Decision Tree Regression to predict null values of column 'age'.

In [43]:
print(df_customers['club_member_status'].unique())
print(df_customers['fashion_news_frequency'].unique())

['ACTIVE' nan 'PRE-CREATE' 'LEFT CLUB']
['NONE' 'Regularly' nan 'Monthly']


In [44]:
df_customers=pd.concat([pd.get_dummies(df_customers[['club_member_status', 'fashion_news_frequency']]),df_customers[['customer_id','age']]], axis=1)

In [45]:
df_customers.head(5)

,club_member_status_ACTIVE,club_member_status_LEFT CLUB,club_member_status_PRE-CREATE,fashion_news_frequency_Monthly,fashion_news_frequency_NONE,fashion_news_frequency_Regularly,customer_id,age
0,1,0,0,0,1,0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,49.0
1,1,0,0,0,1,0,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,25.0
2,1,0,0,0,1,0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,24.0
3,1,0,0,0,1,0,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,54.0
4,1,0,0,0,0,1,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,52.0


#### Fill Nulls Method 1, Linear Regression

In [47]:
testdf = df_customers[df_customers['age'].isnull()==True]
traindf = df_customers[df_customers['age'].isnull()==False]
traindf_1 = df_customers[df_customers['age'].isnull()==False]
testdf.drop(["age","customer_id"],axis=1,inplace=True)
traindf.drop(["age","customer_id"],axis=1,inplace=True)

y = df_customers[df_customers['age'].isnull()==False].drop(columns=['club_member_status_ACTIVE', 'club_member_status_LEFT CLUB','club_member_status_PRE-CREATE','fashion_news_frequency_Monthly','fashion_news_frequency_NONE','fashion_news_frequency_Regularly','customer_id'])
Max = np.max(y['age'])
Min = np.min(y['age'])
#y['age']=(y['age']-Min)/(Max-Min)

/tmp/ipykernel_49725/1770032918.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf.drop(["age","customer_id"],axis=1,inplace=True)
/tmp/ipykernel_49725/1770032918.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  traindf.drop(["age","customer_id"],axis=1,inplace=True)


In [48]:
imputer = SimpleImputer(strategy='mean')
regressor = Pipeline(
steps=[('imputer', imputer), ('regressor', LinearRegression())]
)
scores1 = cross_val_score(regressor, traindf, y, cv=10, scoring='neg_mean_squared_error')
scores2 = cross_val_score(regressor, traindf, y, cv=10, scoring='neg_mean_absolute_error')
scores3 = cross_val_score(regressor, traindf, y, cv=10, scoring='r2')

print("mse is " + str(scores1.mean())+ " std is " + str(scores1.std()))
print("mae is " + str(scores2.mean())+ " std is " + str(scores2.std()))
print("r2 is " + str(scores3.mean())+ " std is " + str(scores3.std()))

mse is -203.02547291078378 std is 0.6660940037534767
mae is -12.387169015046094 std is 0.022423662668823546
r2 is 0.009046921292606359 std is 0.00043805181167022404


#### Fill Nulls Method 2, Decision Tree Regression

In [49]:
kf = KFold(n_splits=100)
models_dtr = dict()
models_dtr_mse=[]
models_dtr_mae=[]
models_dtr_r2_score=[]
i=0
for train_index, test_index in kf.split(traindf):  
    x_train, x_test = traindf.iloc[train_index], traindf.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    #Decision Tree Regression model
    model_dtr = DecisionTreeRegressor(random_state = 0) 
    model_dtr.fit(x_train, y_train)
    predict_dtr=model_dtr.predict(x_test)
    predict_dtr_train=model_dtr.predict(x_train)
    models_dtr_mse.append(mean_squared_error(predict_dtr,y_test))
    models_dtr_mae.append(mean_absolute_error(predict_dtr,y_test))
    models_dtr_r2_score.append(r2_score(predict_dtr_train,y_train))
    models_dtr['model_dtr_{}'.format(i)] = model_dtr
    i+=1

#get the best performance fold, which reaches the min MSE
models_dtr_mse_list_minvalue = min(models_dtr_mse) 
models_dtr_mse_list_minindex = models_dtr_mse.index(min(models_dtr_mse))
print(models_dtr_mse_list_minvalue)
#get the best performance fold, which reaches the min MAE
models_dtr_mae_list_minvalue = min(models_dtr_mae)
models_dtr_mae_list_minindex = models_dtr_mae.index(min(models_dtr_mae)) 
print(models_dtr_mae_list_minvalue)
#get the best performance fold, which reaches the max r2_score
models_dtr_r2_score_list_maxvalue = max(models_dtr_r2_score)
models_dtr_r2_score_list_maxindex = models_dtr_r2_score.index(max(models_dtr_r2_score)) 
print(models_dtr_r2_score_list_maxvalue)

198.55994977163093
12.223805367042202
-104.2275252167523


* However, the result above shows that neither linear regression model nor decision tree regression model achieves good performance.
* Thus we decide to fill the 'age' null values with the mean value.

In [55]:
testdf['age']=y['age'].mean()

/tmp/ipykernel_49725/503012301.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  testdf['age']=y['age'].mean()


* Scale the column 'age' via Min-max method

In [57]:
df_customers_age_predict=pd.concat([testdf['age'],df_customers[df_customers['age'].isnull()==False]['age']],axis=0).sort_index()
df_customers_forFinalMerge.drop('age', axis=1, inplace=True)
df_customers_FinalMerge=pd.concat([df_customers_forFinalMerge,df_customers_age_predict],axis=1)
df_customers_FinalMerge['age']=(df_customers_FinalMerge['age']-Min)/(Max-Min)

In [59]:
df_customers_FinalMerge.head(5)

,customer_id,club_member_status,fashion_news_frequency,age
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,ACTIVE,NONE,0.397590
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,ACTIVE,NONE,0.108434
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,ACTIVE,NONE,0.096386
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,ACTIVE,NONE,0.457831
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,ACTIVE,Regularly,0.433735


### simplify the representation of customer_id

In [60]:
customer_ids = df_customers_FinalMerge['customer_id'].values
customerid_dict = {} 
for idx, customer_id in enumerate(customer_ids):
    customerid_dict.update({customer_id: idx})
df_customers_FinalMerge["customer_id"]= df_customers_FinalMerge["customer_id"].map(lambda x: customerid_dict[x]) 

In [61]:
df_customers_FinalMerge.head(5)

,customer_id,club_member_status,fashion_news_frequency,age
0,0,ACTIVE,NONE,0.397590
1,1,ACTIVE,NONE,0.108434
2,2,ACTIVE,NONE,0.096386
3,3,ACTIVE,NONE,0.457831
4,4,ACTIVE,Regularly,0.433735


### Output the preproecssed result for customers.csv

In [62]:
df_customers_FinalMerge.to_csv('Customer_preprocessed.csv')

# 3.2 Articles.csv

## 3.2.1 data set overview

In [4]:
df_articles=pd.read_csv('articles.csv',header=0)
df_articles.head(5)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [5]:
print(df_articles.shape)
print(preProcessing.uniqueValues(df_articles))
print(preProcessing.isNull(df_articles))
df_articles['detail_desc'].replace('','NONE',inplace=True)

(105542, 25)
article_id int64 has 105542 unique values
product_code int64 has 47224 unique values
prod_name object has 45875 unique values
product_type_no int64 has 132 unique values
product_type_name object has 131 unique values
product_group_name object has 19 unique values
graphical_appearance_no int64 has 30 unique values
graphical_appearance_name object has 30 unique values
colour_group_code int64 has 50 unique values
colour_group_name object has 50 unique values
perceived_colour_value_id int64 has 8 unique values
perceived_colour_value_name object has 8 unique values
perceived_colour_master_id int64 has 20 unique values
perceived_colour_master_name object has 20 unique values
department_no int64 has 299 unique values
department_name object has 250 unique values
index_code object has 10 unique values
index_name object has 10 unique values
index_group_no int64 has 5 unique values
index_group_name object has 5 unique values
section_no int64 has 57 unique values
section_name object h

## 3.2.2 Implement the clip with column 'detail_desc' with Image set to generate text-image features


* we found that the column 'detail_desc' is a paragraph
* it will be processed with images together using CLIP method. 
* CLIP is used to generate text-image combined features. 
* the result into articles_preprocessed.csv.

In [6]:
df_articles_clip=pd.read_csv('articles_clipped.csv',header=0)

In [7]:
df_articles_clip.head(5)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,detail_desc_502,detail_desc_503,detail_desc_504,detail_desc_505,detail_desc_506,detail_desc_507,detail_desc_508,detail_desc_509,detail_desc_510,detail_desc_511
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,0.242554,0.030151,0.773926,-0.138916,-0.055878,0.079285,0.163696,-0.125854,-0.140869,0.192261
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,0.242554,0.030151,0.773926,-0.138916,-0.055878,0.079285,0.163696,-0.125854,-0.140869,0.192261
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,0.242554,0.030151,0.773926,-0.138916,-0.055878,0.079285,0.163696,-0.125854,-0.140869,0.192261
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,-0.428223,0.119812,0.293945,-0.357910,0.044708,0.007214,-0.014244,0.333252,0.100525,-0.125122
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,-0.428223,0.119812,0.293945,-0.357910,0.044708,0.007214,-0.014244,0.333252,0.100525,-0.125122


## 3.2.3 Implement PCA to reduce dimensions for text-image features

In [11]:
df_img_arttributes =df_articles_clip.iloc[:, 24:24+512]

In [12]:
df_img_arttributes.head()

,image_0,image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,image_9,...,image_502,image_503,image_504,image_505,image_506,image_507,image_508,image_509,image_510,image_511
0,-0.004272,0.080017,-0.033997,0.356689,0.113098,0.410645,0.177246,-0.045624,0.601074,0.267334,...,-0.267334,0.010338,0.449463,-0.349121,-0.109070,0.163574,-0.122742,0.967773,-0.500977,0.480225
1,-0.208862,0.120667,0.039032,0.386475,-0.031143,0.304688,0.132935,-0.006721,0.646484,0.241943,...,-0.136108,-0.111023,0.499756,-0.218506,0.046814,0.076172,-0.225952,0.834961,-0.677246,0.201294
2,-0.308105,0.135986,0.018936,0.392334,0.129272,0.400391,0.250244,-0.126465,0.535156,0.199951,...,-0.201660,0.028030,0.506348,-0.406006,-0.078552,0.220703,-0.263916,0.851562,-0.703613,0.387451
3,0.200195,-0.054047,-0.098145,-0.214600,-0.339111,-0.030930,0.196655,-0.028717,0.429688,-0.306152,...,-0.141846,-0.101257,0.316406,-0.082764,-0.209717,0.043884,0.041809,0.433594,-0.438477,0.261475
4,-0.274414,0.010376,0.262939,-0.354248,-0.598145,0.028152,0.169434,-0.047913,0.443604,-0.287109,...,-0.062164,0.040466,0.379639,-0.158691,-0.164795,0.014374,0.021484,0.202881,-0.345703,0.043701


In [13]:
df_img_arttributes.shape

(105543, 512)

In [14]:
pca_img = PCA(n_components=20)
df_img_arttributes_pca = pca_img.fit_transform(df_img_arttributes)

In [15]:
img_column_names=copy.deepcopy(df_articles_clip.columns[24: 24+20].values.tolist())

In [17]:
df_img_arttributes_pca=pd.DataFrame(df_img_arttributes_pca,columns=img_column_names)

In [18]:
df_img_arttributes_pca.head()

,image_0,image_1,image_2,image_3,image_4,image_5,image_6,image_7,image_8,image_9,image_10,image_11,image_12,image_13,image_14,image_15,image_16,image_17,image_18,image_19
0,-0.904951,-0.649637,0.446407,-0.010903,0.186668,2.484099,0.258305,-0.871315,0.834459,0.523042,-1.319842,1.016106,0.121072,-0.676786,-0.342197,-0.625527,-0.402196,0.046940,-0.241220,-0.225766
1,-0.423777,-0.002506,-0.002448,0.684817,-0.235998,2.412108,-0.184227,-1.085126,1.341054,-0.210054,-0.298375,-0.019754,0.013886,-1.258512,-0.543482,-1.049890,-0.083958,0.456676,0.176678,-0.252001
2,-0.333081,0.369323,-0.155064,-0.148885,-0.585058,2.253307,0.277516,-0.510558,1.949587,-0.785743,-1.647060,0.345464,0.250877,-0.871449,-0.579443,-0.352076,0.743831,0.468776,0.191003,-0.177249
3,1.962173,-1.484765,1.263352,-0.435849,2.264806,2.591253,-0.398836,-0.259645,-1.345897,0.475352,-0.472802,0.394598,-0.115400,-0.984386,1.322717,-0.406187,0.419705,0.361761,-0.990038,-0.426614
4,2.897516,-1.209935,0.789383,-0.510141,2.133664,2.425823,-1.669747,-0.134036,-0.582452,0.547219,1.079926,-0.360890,0.007037,-1.151344,0.927302,-0.069287,0.986330,0.590559,-0.884532,-0.429165


In [19]:
pca_img.explained_variance_ratio_

array([0.08438521, 0.0681501 , 0.0646505 , 0.04993961, 0.03577513,
       0.03527892, 0.03204195, 0.02654545, 0.02279742, 0.02102092,
       0.01900102, 0.01861094, 0.01774867, 0.0165459 , 0.01524588,
       0.01462324, 0.01353792, 0.01261257, 0.01227662, 0.01129376])

In [20]:
pca_img.singular_values_

array([503.1376455 , 452.15453289, 440.3921909 , 387.05815943,
       327.60028313, 325.32039072, 310.03669665, 282.19458158,
       261.51492851, 251.11897451, 238.74929548, 236.28592414,
       230.74729591, 222.79161791, 213.86016347, 209.44760494,
       201.52535122, 194.5160421 , 191.90801074, 184.06570337])

In [21]:
df_desc_arttributes =df_articles_clip.iloc[:, 24+512:24+512+512]

In [22]:
df_desc_arttributes.head()

,detail_desc_0,detail_desc_1,detail_desc_2,detail_desc_3,detail_desc_4,detail_desc_5,detail_desc_6,detail_desc_7,detail_desc_8,detail_desc_9,...,detail_desc_502,detail_desc_503,detail_desc_504,detail_desc_505,detail_desc_506,detail_desc_507,detail_desc_508,detail_desc_509,detail_desc_510,detail_desc_511
0,0.254395,0.030029,-0.429688,0.281250,-0.123230,0.562012,-0.159302,-1.347656,0.479980,0.361816,...,0.242554,0.030151,0.773926,-0.138916,-0.055878,0.079285,0.163696,-0.125854,-0.140869,0.192261
1,0.254395,0.030029,-0.429688,0.281250,-0.123230,0.562012,-0.159302,-1.347656,0.479980,0.361816,...,0.242554,0.030151,0.773926,-0.138916,-0.055878,0.079285,0.163696,-0.125854,-0.140869,0.192261
2,0.254395,0.030029,-0.429688,0.281250,-0.123230,0.562012,-0.159302,-1.347656,0.479980,0.361816,...,0.242554,0.030151,0.773926,-0.138916,-0.055878,0.079285,0.163696,-0.125854,-0.140869,0.192261
3,0.202759,0.366455,-0.447510,-0.110474,-0.588867,0.224243,0.123047,-0.300781,0.685547,0.029541,...,-0.428223,0.119812,0.293945,-0.357910,0.044708,0.007214,-0.014244,0.333252,0.100525,-0.125122
4,0.202759,0.366455,-0.447510,-0.110474,-0.588867,0.224243,0.123047,-0.300781,0.685547,0.029541,...,-0.428223,0.119812,0.293945,-0.357910,0.044708,0.007214,-0.014244,0.333252,0.100525,-0.125122


In [23]:
df_desc_arttributes.shape

(105543, 512)

In [24]:
pca_desc = PCA(n_components=20)
df_desc_arttributes_pca = pca_desc.fit_transform(df_desc_arttributes)

In [25]:
desc_column_names = copy.deepcopy(df_articles_clip.columns[24+512: 24+512+20].values.tolist())

In [26]:
df_desc_arttributes_pca=pd.DataFrame(df_desc_arttributes_pca,columns=desc_column_names)
df_desc_arttributes_pca.head()

,detail_desc_0,detail_desc_1,detail_desc_2,detail_desc_3,detail_desc_4,detail_desc_5,detail_desc_6,detail_desc_7,detail_desc_8,detail_desc_9,detail_desc_10,detail_desc_11,detail_desc_12,detail_desc_13,detail_desc_14,detail_desc_15,detail_desc_16,detail_desc_17,detail_desc_18,detail_desc_19
0,-1.486972,1.055092,-0.371724,-1.232335,-1.655621,0.318552,0.801550,-0.081535,-1.383661,-0.752586,-0.585763,0.951265,0.457601,0.381208,-0.012477,-0.832238,0.168741,0.918360,-0.413755,-0.380835
1,-1.486972,1.055092,-0.371724,-1.232335,-1.655621,0.318552,0.801550,-0.081535,-1.383661,-0.752586,-0.585763,0.951265,0.457601,0.381208,-0.012477,-0.832238,0.168741,0.918360,-0.413755,-0.380835
2,-1.486972,1.055092,-0.371724,-1.232335,-1.655621,0.318552,0.801550,-0.081535,-1.383661,-0.752586,-0.585763,0.951265,0.457601,0.381208,-0.012477,-0.832238,0.168741,0.918360,-0.413755,-0.380835
3,0.383950,1.463732,-0.096874,2.857092,-1.378807,0.780725,0.187752,-1.079697,-0.502805,-0.979120,-0.594991,-0.174220,0.285715,0.020980,-0.326465,0.580976,-0.346438,-0.912049,-0.213008,0.297638
4,0.383950,1.463732,-0.096874,2.857092,-1.378807,0.780725,0.187752,-1.079697,-0.502805,-0.979120,-0.594991,-0.174220,0.285715,0.020980,-0.326465,0.580976,-0.346438,-0.912049,-0.213008,0.297638


In [27]:
pca_desc.explained_variance_ratio_

array([0.10126429, 0.07606767, 0.06827015, 0.05107753, 0.04598785,
       0.03903326, 0.03662329, 0.03097685, 0.02852022, 0.02318894,
       0.02232897, 0.02159645, 0.01836077, 0.01778292, 0.0174576 ,
       0.01427361, 0.01370867, 0.01223   , 0.01176754, 0.01086572])

In [28]:
pca_desc.singular_values_

array([493.03611416, 427.31745221, 404.8237744 , 350.15918305,
       332.25545924, 306.1032812 , 296.5031276 , 272.69000102,
       261.65378694, 235.93428601, 231.51807689, 227.68884155,
       209.94040605, 206.61038458, 204.71181022, 185.10471277,
       181.40456488, 171.34196897, 168.07126448, 161.50267927])

## 3.2.4 Drop original data set's uneccessary columns

In [29]:
df_articles_orignal=df_articles_clip.iloc[:, 0:24]
df_articles_orignal.head(5)

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,1676,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear"
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,1339,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear"


* Some pair columns are of same meaning, for example, column 'product_type_no' and 'product_type_name' are exactly same meaning with one number-category and one string-category. 
* We choose to reserve the string format, to eliminate the risk that later the tool Recbole will mistreat the number-category as numbers.
* that's why we will delete columns 'product_code','product_type_no','graphical_appearance_no','colour_group_code','perceived_colour_value_id', \n'perceived_colour_master_id','department_no','index_code','index_group_no','section_no','garment_group_no'

* Some colunms are subclass of other columns, for example, 'product_type_name' and 'product_group_name', they express the same meaning but one with higher granularity.
* In this case, we choose to delete the parent group columns, to retain the granularity and cancel the duplicates.
* That's why we will delete columns 'product_group_name','index_group_name','garment_group_name'.

* Some columns are the similar representation of other columns, for example 'prod-
uct_type_name' has similar meaning as ’department_name’, thus ’department_name’ will be
deleted.

In [30]:
df_articles_orignal.drop(columns=['product_code','prod_name','product_type_no','product_group_name','graphical_appearance_no','colour_group_code','perceived_colour_value_id','perceived_colour_master_id','department_no','department_name','index_code','index_group_no','index_group_name','section_no','garment_group_no','garment_group_name'],inplace=True)

In [31]:
df_articles_orignal.head(5)

,article_id,product_type_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,index_name,section_name
0,108775015,Vest top,Solid,Black,Dark,Black,Ladieswear,Womens Everyday Basics
1,108775044,Vest top,Solid,White,Light,White,Ladieswear,Womens Everyday Basics
2,108775051,Vest top,Stripe,Off White,Dusty Light,White,Ladieswear,Womens Everyday Basics
3,110065001,Bra,Solid,Black,Dark,Black,Lingeries/Tights,Womens Lingerie
4,110065002,Bra,Solid,White,Light,White,Lingeries/Tights,Womens Lingerie


## 3.2.5 Merge original attribues with Clipped text-image dimension-reduced features

In [33]:
articles_output=pd.concat([df_articles_orignal,df_img_arttributes_pca,df_desc_arttributes_pca],axis=1)
articles_output.head()

,article_id,product_type_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,index_name,section_name,image_0,image_1,...,detail_desc_10,detail_desc_11,detail_desc_12,detail_desc_13,detail_desc_14,detail_desc_15,detail_desc_16,detail_desc_17,detail_desc_18,detail_desc_19
0,108775015,Vest top,Solid,Black,Dark,Black,Ladieswear,Womens Everyday Basics,-0.904951,-0.649637,...,-0.585763,0.951265,0.457601,0.381208,-0.012477,-0.832238,0.168741,0.918360,-0.413755,-0.380835
1,108775044,Vest top,Solid,White,Light,White,Ladieswear,Womens Everyday Basics,-0.423777,-0.002506,...,-0.585763,0.951265,0.457601,0.381208,-0.012477,-0.832238,0.168741,0.918360,-0.413755,-0.380835
2,108775051,Vest top,Stripe,Off White,Dusty Light,White,Ladieswear,Womens Everyday Basics,-0.333081,0.369323,...,-0.585763,0.951265,0.457601,0.381208,-0.012477,-0.832238,0.168741,0.918360,-0.413755,-0.380835
3,110065001,Bra,Solid,Black,Dark,Black,Lingeries/Tights,Womens Lingerie,1.962173,-1.484765,...,-0.594991,-0.174220,0.285715,0.020980,-0.326465,0.580976,-0.346438,-0.912049,-0.213008,0.297638
4,110065002,Bra,Solid,White,Light,White,Lingeries/Tights,Womens Lingerie,2.897516,-1.209935,...,-0.594991,-0.174220,0.285715,0.020980,-0.326465,0.580976,-0.346438,-0.912049,-0.213008,0.297638


In [35]:
articles_output.to_csv('Articles_preprocessed.csv', index=None)

## 3.3 Transactions_train.csv

In [63]:
df_transactions_train=pd.read_csv('transactions_train.csv',header=0)
print(df_transactions_train.shape)
print(preProcessing.isNull(df_transactions_train))
print(preProcessing.uniqueValues(df_transactions_train))
df_transactions_train.head(5)

(31788324, 5)
Series([], dtype: int64)
t_dat object has 734 unique values
customer_id object has 1362281 unique values
article_id int64 has 104547 unique values
price float64 has 9857 unique values
sales_channel_id int64 has 2 unique values
None


,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


### Change the time to epoch
* since the tool Recbole requires time to be epoch format

In [64]:
df_transactions_train['t_dat']=pd.to_datetime(df_transactions_train.t_dat, format='%Y-%m-%d')
df_transactions_train['t_dat']=(df_transactions_train['t_dat']-dt.datetime(1970,1,1)).dt.total_seconds()

### Scale the column 'price' using min-max

In [65]:
scaler_price = MinMaxScaler()
df_transactions_train['price'] = scaler_price.fit_transform(df_transactions_train['price'].to_frame())

### Simplify the representation of customer_id

In [66]:
df_transactions_train["customer_id"]= df_transactions_train["customer_id"].map(lambda x: customerid_dict[x]) 

### Add label column

* In Recbole, the transaction interaction indicators are always defined as 'ratings', 'number of clicks', etc.
* Our dataset has no such indicators.
* Thus we create mannually a column, named 'interaction'. Each row is a buy behavior, thus the 'interaction' column value is assigned to 1.

In [67]:
df_transactions_train['interaction']  = 1.0 

In [68]:
df_transactions_train.head()

,t_dat,customer_id,article_id,price,sales_channel_id,interaction
0,1.537402e+09,2,663713001,0.085905,2,1.0
1,1.537402e+09,2,541518023,0.051520,2,1.0
2,1.537402e+09,7,505221004,0.025731,2,1.0
3,1.537402e+09,7,685687003,0.028597,2,1.0
4,1.537402e+09,7,685687004,0.028597,2,1.0


In [ ]:
df_transactions_train.to_csv('Transactions_train_preprocessed.csv', index=None)